In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import random

from IPython.display import display

from nltk import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.dummy import DummyClassifier
import nltk

In [ ]:
train_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/train.csv'
df_train = pd.read_csv(train_url)
x_train = df_train['review_es']
y_train = df_train['sentimiento']
test_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/test.csv'
df_test = pd.read_csv(test_url)
x_test = df_test['review_es']

In [ ]:
display(df_train)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo
...,...,...,...
49995,49995,Pensé que esta película hizo un buen trabajo a...,positivo
49996,49996,"Mala parcela, mal diálogo, mala actuación, dir...",negativo
49997,49997,Soy católica enseñada en escuelas primarias pa...,negativo
49998,49998,Voy a tener que estar en desacuerdo con el com...,negativo


Ver si hay campos nulos, cantidad de criticas y el resultado de las criticas.

In [ ]:
print("Cantidad de criticas cinematograficas: ", df_train.shape[0])
print("")

display(df_train.isna().sum())
print("")

print("Ver cuantas criticas son positivas y negativas")
display(df_train['sentimiento'].value_counts())

Cantidad de criticas cinematograficas:  50000



ID             0
review_es      0
sentimiento    0
dtype: int64


Ver cuantas criticas son positivas y negativas


sentimiento
positivo    25000
negativo    25000
Name: count, dtype: int64

#Preprocesamiento

##Dectentor de idioma

In [ ]:
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=34cc598eb8cd84c1baf94a79d6dde5aa76447d009c78f07d29965e67f83d8c35
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [ ]:
import langid

In [ ]:
def detect_language(text):
    return langid.classify(text)[0]

In [ ]:
df_train['idioma'] = df_train['review_es'].apply(detect_language)

In [ ]:
df_train = df_train[df_train['idioma'] == 'es']

##Limpieza del texto

In [ ]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 47.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.0 MB/s eta 0:00:00


In [ ]:
from unidecode import unidecode
import re
import spacy

In [ ]:
def limpiar(texto):
  texto = texto.lower() #todo a minusculas
  texto = unidecode(texto) #eliminar acentos
  texto = re.sub(r'[^a-zA-Z\s]', '', texto)#Elimina todos los caracteres que no sean letras (de la A a la Z, sin importar mayúsculas o minúsculas) o espacios.
  texto = re.sub(r'\W', ' ', str(texto))#Reemplaza cualquier carácter no alfanumérico con un espacio.
  texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', texto)#Elimina palabras que consisten en un solo carácter.
  texto = re.sub(r'\s+', ' ', texto, flags=re.I)#Reemplaza múltiples espacios en blanco consecutivos con un solo espacio
  return texto

In [ ]:
df_train['review_es'] = df_train['review_es'].apply(limpiar)

<ipython-input-16-00b2c10f3747>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['review_es'] = df_train['review_es'].apply(limpiar)


In [ ]:
df_test['review_es'] = df_test['review_es'].apply(limpiar)

In [ ]:
df_train.to_csv('df_train_limpiado.csv')
df_test.to_csv('df_test_limpiado.csv')

In [ ]:
df_train

,ID,review_es,sentimiento,idioma
0,0,uno de los otros criticos ha mencionado que de...,positivo,es
1,1,una pequena pequena produccionla tecnica de fi...,positivo,es
2,2,pense que esta era una manera maravillosa de p...,positivo,es
3,3,basicamente hay una familia donde un nino pequ...,negativo,es
4,4,el amor en el tiempo de petter mattei es una p...,positivo,es
...,...,...,...,...
49995,49995,pense que esta pelicula hizo un buen trabajo l...,positivo,es
49996,49996,mala parcela mal dialogo mala actuacion direcc...,negativo,es
49997,49997,soy catolica ensenada en escuelas primarias pa...,negativo,es
49998,49998,voy tener que estar en desacuerdo con el comen...,negativo,es


##Insertar stop words

In [ ]:
!pip install stop_words
import stop_words
# Usamos stop words del español
stop_words = stop_words.get_stop_words('es')

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=38eb1cde4866f71ec34d0b6539aca06c85d143fed3211b2b79f0bdb7f9ac4cae
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [ ]:
display(stop_words)

['a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

Eliminar stop words del texto

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Pre: todas las palabras deben de estar en minuscula
def eliminar_palabras(text):
    texto_tokenizado = word_tokenize(text) #Tokenizar texto
    texto_filtrado = [palabra for palabra in texto_tokenizado if palabra not in stop_words]
    return ' '.join(texto_filtrado)

In [ ]:
df_train['review_es'] = df_train['review_es'].apply(eliminar_palabras)

In [ ]:
df_train.to_csv('df_train_limpiado_stopWords.csv')

In [ ]:
df_test['review_es'] = df_test['review_es'].apply(eliminar_palabras)

In [ ]:
df_test.to_csv('df_test_limpiado_stopWords.csv')

In [ ]:
df_train

,ID,review_es,sentimiento,idioma
0,0,criticos mencionado despues ver solo oz episod...,positivo,es
1,1,pequena pequena produccionla tecnica filmacion...,positivo,es
2,2,pense manera maravillosa pasar tiempo fin sema...,positivo,es
3,3,basicamente familia nino pequeno jake piensa z...,negativo,es
4,4,amor tiempo petter mattei pelicula visualmente...,positivo,es
...,...,...,...,...
49995,49995,pense pelicula hizo buen trabajo derechano tan...,positivo,es
49996,49996,mala parcela mal dialogo mala actuacion direcc...,negativo,es
49997,49997,catolica ensenada escuelas primarias parroquia...,negativo,es
49998,49998,voy tener desacuerdo comentario anterior lado ...,negativo,es


##Lematizar texto

In [ ]:
nlp = spacy.load('es_core_news_sm')

In [ ]:
def lematizar(texto):
    doc = nlp(texto)#pasa el texto por el modelo de spaCy, devolviendo un objeto doc que contiene una lista de tokens.
    lemas = [token.lemma_ for token in doc]#Extrae el lema de cada token (palabra) en cada doc
    return ' '.join(lemas)#une todos los Lemas del doc en un Solo Texto separados por un espaio

In [ ]:
df_train['review_es'] = df_train['review_es'].apply(lematizar)

In [ ]:
df_train.to_csv('df_train_preprocesado.csv')

In [ ]:
df_test['review_es'] = df_test['review_es'].apply(lematizar)

In [ ]:
df_test.to_csv('df_test_preprocesado.csv')

In [ ]:
df_train

,ID,review_es,sentimiento,idioma
0,0,critico mencionado despu ver solo oz episodio ...,positivo,es
1,1,pequén pequén produccionla tecnico filmacion i...,positivo,es
2,2,pense manera maravilloso pasar tiempo fin sema...,positivo,es
3,3,basicamente familia nino pequeno jake pensar z...,negativo,es
4,4,amor tiempo petter mattei pelicular visualment...,positivo,es
...,...,...,...,...
49995,49995,pense peliculo hacer buen trabajo derechano ta...,positivo,es
49996,49996,malo parcelar mal dialogo malo actuacion direc...,negativo,es
49997,49997,catolico ensenado escuela primario parroquial ...,negativo,es
49998,49998,ir tener desacuerdo comentario anterior lado m...,negativo,es
